In [ ]:
import pandas as pd
import numpy as np
import os
from pixelmed_calc.medical_imaging.RadiologyComponents.solution import convert_chinese_columns_to_numeric
os.makedirs('results/img', exist_ok=True)
os.makedirs('results/model_weight', exist_ok=True)
os.makedirs('results/pred', exist_ok=True)
inputfile = 'convert_chinese_columns_to_numeric.csv'
labelfile='group.csv'
data_feature=pd.read_csv(inputfile)
categorical = ['是否溢液']
# 转换数值
data_feature = convert_chinese_columns_to_numeric(data_feature, categorical)
mean_values = data_feature.loc[:, data_feature.columns != 'ID'].mean()
data_feature.fillna(mean_values, inplace=True)
data_label=pd.read_csv(labelfile)
merged_data = pd.merge(data_feature, data_label, on='ID')
merged_data.columns = merged_data.columns.str.replace(r'-', '_')
merged_data

数据集包含较多的异常值，使用稳键归一化（robust normalization）
value_result = (value-Media)/(Q1-Q3)  
Q1的位置 = 1 * （n + 1) / 4  
Q3的位置 =  3 *（n + 1) / 4  
n : 表示数据的个数。  
media : 中位数  
Q1 : 是第 1 个四分位数（第 25 个分位数）  
Q3 : 第 3 个四分位数（第 75 个分位数）  


In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler_cloum = merged_data.columns[1:-2]
merged_data[scaler_cloum]=scaler.fit_transform(merged_data[scaler_cloum])
merged_data

## F检验筛选特征

多组比较：F检验通常用于分析方差分析（ANOVA），它可以同时比较三个或更多的组别。而T检验通常用于两组数据的比较。 
 方差齐性检验：在进行T检验之前，通常需要先检验数据的方差是否齐性。F检验可以直接用于检验方差是否相等，这是T检验无法做到的。  
 控制类型I错误：在进行多重比较的情况下，T检验可能会增加犯类型I错误的风险。而F检验通过调整P值，可以更好地控制这一风险。  
 更广泛的适用性：F检验不仅限于两样本均值的比较，它还适用于多个样本均值的比较，以及因子水平的比较。  
 实验设计：F检验可以应用于完全随机设计、随机区组设计、拉丁方设计等多种实验设计，而T检验通常只适用于两组独立样本或配对样本。  
 多重比较问题：当涉及到多个比较时，F检验可以通过调整方法（如Bonferroni校正）来控制整体错误率，而T检验在多重比较时可能需要更复杂的方法来控制错误率。  
 统计功效：在某些情况下，F检验可能具有比T检验更高的统计功效，尤其是在样本量较大时。  
 非正态分布数据：虽然T检验和F检验都假设数据正态分布，但在实际应用中，F检验对于数据分布的正态性要求可能稍微宽松一些。  

In [ ]:
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import select_significant_features

significant_features_sel, feature_scores=select_significant_features(merged_data, label_column='label', columns=merged_data.columns[1:-2],significance_level=0.7,method='f_test')
feature_scores

In [ ]:
significant_features_sel

In [ ]:
# 计算 Pearson 相关系数矩阵
correlation_matrix = merged_data[significant_features_sel].corr('pearson')

# 标记高相关性的特征对
high_corr_pairs = np.where(np.abs(correlation_matrix) > 0.7)
high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y]) for x, y in zip(*high_corr_pairs) if x != y and x < y]

# 创建一个集合来存储需要移除的特征
features_to_remove = set()

# 遍历高相关性特征对，添加一个特征到需要移除的集合中
for (feature1, feature2) in high_corr_pairs:
    features_to_remove.add(feature2)

# 创建新的显著特征列表，移除高相关性的特征
final_significant_features = [feature for feature in significant_features_sel if feature not in features_to_remove]

# 输出最终的特征列表
print(f"最终筛选的特征数量: {len(final_significant_features)}")
print(final_significant_features)

# 输出需要移除的特征
print(f"移除的特征: {features_to_remove}")

# 将最终筛选的特征数据保存到 merged_data_final
#merged_data_final = merged_data[['ID', 'label'] + final_significant_features]

In [ ]:
data_sel=merged_data[list(final_significant_features)+['label','group']]
data_sel

In [7]:
x_train=data_sel[data_sel['group']=='train'].drop(['group','label'],axis=1)
y_train=data_sel[data_sel['group']=='train']['label']

x_val=data_sel[data_sel['group']=='val'].drop(['group','label'],axis=1)
y_val=data_sel[data_sel['group']=='val']['label']

x_test=data_sel[data_sel['group']=='test'].drop(['group','label'],axis=1)
y_test=data_sel[data_sel['group']=='test']['label']

x_all=data_sel.drop(['group','label'],axis=1)
y_all=data_sel['label']

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import find_best_alpha
alphas = np.logspace(-6,2,50)
best_alpha,scores_lasso,scores_std_lasso=find_best_alpha(x_train, y_train,alphas=alphas)

lasso = Lasso(alpha=best_alpha)
lasso.fit(x_train, y_train)
# 输出系数
lasso_coefs = lasso.coef_
# 创建 DataFrame 显示特征名和对应的系数
coef_df = pd.DataFrame({'feature': x_train.columns, 'coef': lasso_coefs})
selected_features = coef_df#[abs(coef_df['coef']) > 1e-1]
# 创建 DataFrame 显示特征名和对应的系数
print(f"选择的特征:\n{selected_features}")
lasso_sel_cloumn=selected_features['feature']

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.dpi'] = 300
# 设置不同的alpha值范围
alpha_values = [0.0001,0.001,0.002,0.005,0.008,0.01,0.1,1,10,50,100]#alphas
lasso = Lasso(max_iter=1000)
coefs = []
num_nonzero_coefs = []
for a in alpha_values:
    lasso.set_params(alpha=a)
    lasso.fit(x_train, y_train)
    coefs.append(lasso.coef_)
    num_nonzero_coefs.append(np.sum(lasso.coef_ != 0))
ax = plt.gca()
ax.plot(alpha_values, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel(f'Log({best_alpha:.3f})')
plt.ylabel('Coefficients')
plt.title('Lasso coefficients as a function of alpha')
plt.axvline(x=best_alpha, linestyle='--', color='r')
plt.grid(False)
plt.show()


平均分数计算：在完成所有交叉验证迭代后，GridSearchCV会计算每个参数组合在所有测试集上的平均得分。

In [ ]:

plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores_lasso)
# 显示误差线，显示+/-标准。 分数错误
std_error = scores_std_lasso / np.sqrt(10)
plt.semilogx(alphas, scores_lasso + std_error, 'b--')
plt.semilogx(alphas, scores_lasso - std_error, 'b--')
# alpha = 0.2控制填充颜色的半透明性
plt.fill_between(alphas, scores_lasso + std_error, scores_lasso - std_error, alpha=0.2)
plt.axvline(x=best_alpha, linestyle='--', color='r')
plt.ylabel('CV score +/- std error')
plt.xlabel(f'alpha({best_alpha:.3f})')
plt.axhline(np.max(scores_lasso), linestyle='--', color='.5')
plt.xlim([alphas[0], alphas[-1]])
plt.grid(False)
plt.show()

In [ ]:
x_train=x_train[lasso_sel_cloumn]
x_val=x_val[lasso_sel_cloumn]
x_test=x_test[lasso_sel_cloumn]
x_all=x_all[lasso_sel_cloumn]
x_train.to_csv('results/pred/x_train.csv',index=False)
x_val.to_csv('results/pred/x_val.csv',index=False)    
x_test.to_csv('results/pred/x_test.csv',index=False)
y_train.to_csv('results/pred/y_train.csv',index=False)
y_val.to_csv('results/pred/y_val.csv',index=False)    
y_test.to_csv('results/pred/y_test.csv',index=False)

print(x_train.shape ,x_val.shape,x_test.shape)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
models = [('LR', LogisticRegression(random_state=0)),
          ('NB',GaussianNB()),
          ('linear_SVM', SVC(kernel='linear',class_weight='balanced',probability=True,max_iter=1000)),
          ('poly_SVM',SVC(kernel='poly',class_weight='balanced',probability=True)),
          ('sigmoid_SVM',SVC(kernel='sigmoid',class_weight='balanced',probability=True)),
          ('rbf_SVM',SVC(kernel='rbf',class_weight='balanced',probability=True)),
          ('DT', DecisionTreeClassifier(class_weight='balanced')),
          ('RF', RandomForestClassifier(class_weight='balanced')),
          ('ExtraTree', ExtraTreesClassifier(class_weight='balanced')),
          ('XGBoost', XGBClassifier(class_weight='balanced')),
          ('AdaBoost',AdaBoostClassifier(n_estimators=10, random_state=0)),
          ('MLP',MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=200, solver='adam', random_state=42)),
          ('GBM',GradientBoostingClassifier(n_estimators=10, random_state=0)),
          ('LightGBM',LGBMClassifier(n_estimators=10, max_depth=-1, objective='binary',verbosity=-1))]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('LightGBM').setLevel(logging.ERROR)  # 仅输出错误信息
import joblib
from sklearn.utils import resample
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import calculate_metrics_with_ci

# 创建DataFrame存储结果
results = pd.DataFrame(columns=['Dataset', 'Model', 'Threshold', 'ACC', 'AUC', 'Sensitivity', 'Specificity', 'NPV', 'PPV', 'F1'])
ci_results = pd.DataFrame(columns=['Dataset', 'Model', 'ACC', 'AUC', 'Sensitivity', 'Specificity', 'NPV', 'PPV', 'F1'])

proba_dict_train = {}
proba_dict_test = {}
proba_dict_val={}
train_ids = merged_data[merged_data['group'] == 'train']['ID'].values
val_ids = merged_data[merged_data['group'] == 'val']['ID'].values
test_ids = merged_data[merged_data['group'] == 'test']['ID'].values

# 训练和测试所有模型
for name, model in models:
    print(f"Training {name}...")
    #model.fit(x_all[0:270], y_all[0:270])
    model.fit(x_train, y_train)
    joblib.dump(model, f"results/model_weight/{name}.pkl")
    # 在训练集、验证集和测试集上分别找到最佳阈值
    y_train_proba = model.predict_proba(x_train)[:, 1]
    y_val_proba = model.predict_proba(x_val)[:, 1]
    y_test_proba = model.predict_proba(x_test)[:, 1]

    # 使用最佳阈值进行预测和计算指标
   
    train_metrics,train_ci = calculate_metrics_with_ci(np.array(y_train),  np.array(y_train_proba),n_bootstrap=100)
    val_metrics,val_ci = calculate_metrics_with_ci(np.array(y_val),  np.array(y_val_proba),n_bootstrap=100)
    
    test_metrics,test_ci = calculate_metrics_with_ci(np.array(y_test), np.array(y_test_proba),n_bootstrap=100)

    proba_dict_train[name] = y_train_proba
    proba_dict_val[name] = y_val_proba
    proba_dict_test[name] = y_test_proba

    train_output = pd.DataFrame({'ID': train_ids, 'proba': y_train_proba})
    train_output.to_csv(f'results/pred/{name}_train_proba.csv', index=False)

    val_output = pd.DataFrame({'ID': val_ids, 'proba': y_val_proba})
    val_output.to_csv(f'results/pred/{name}_val_proba.csv', index=False)

    test_output = pd.DataFrame({'ID': test_ids, 'proba': y_test_proba})
    test_output.to_csv(f'results/pred/{name}_test_proba.csv', index=False)
    
    # 将结果保存到DataFrame中
    for dataset, metrics, res_ci,true_labels, pred_proba in zip(['Train','val', 'Test'], 
                                                        [train_metrics,val_metrics, test_metrics], 
                                                         [train_ci,val_ci, test_ci], 
                                                        [y_train,y_val,y_test],
                                                        [y_train_proba,y_val_proba, y_test_proba]):
        result = {
            'Dataset': dataset,
            'Model': name,
            'Threshold': metrics['threshold'],
            'ACC': metrics['accuracy'],
            'AUC': metrics['auc'],
            'Sensitivity': metrics['sensitivity'],
            'Specificity': metrics['specificity'],
            'NPV': metrics['npv'],
            'PPV': metrics['ppv'],
            'F1': metrics['f1'],
        }
        results = pd.concat([results, pd.DataFrame([result])], ignore_index=True)
        # 计算 95% CI
        ci_data = {
            'Dataset': dataset,
            'Model': name,
            'ACC': res_ci['accuracy'],
            'AUC': res_ci['auc'],
            'Sensitivity': res_ci['sensitivity'],
            'Specificity': res_ci['specificity'],
            'NPV': res_ci['npv'],
            'PPV': res_ci['ppv'],
            'F1': res_ci['f1'],
        }
        ci_results = pd.concat([ci_results, pd.DataFrame([ci_data])], ignore_index=True)
# 显示结果
display(results)
# 保存模型性能指标和95%置信区间到CSV文件
results.to_csv('results/model_performance_metrics.csv', index=False)
ci_results.to_csv('results/model_performance_metrics_CI.csv', index=False)

In [ ]:
for name, model in models:
    if hasattr(model, 'feature_importances_') or hasattr(model, 'coef_'):
        print(f"Plotting feature importance for {name}...")
        feature_importance = None
        if hasattr(model, 'feature_importances_'):
            feature_importance = model.feature_importances_
        elif hasattr(model, 'coef_'):
            feature_importance = np.abs(model.coef_[0])
        
        feature_importance_df = pd.DataFrame({
            'Feature': x_train.columns,
            'Importance': feature_importance
        })
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        
        plt.figure(figsize=(10, 6))
        plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{name} Feature Importance")
        plt.gca().invert_yaxis()
        plt.grid(False)
        plt.savefig(f"results/img/{name}_feature_importance.svg", bbox_inches='tight')
        plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_multiple_ROCs
  
for name in proba_dict_test:
    y_truths_list=[y_train,y_val,y_test]
    y_scores_list=[proba_dict_train[name],proba_dict_val[name],proba_dict_test[name]]
    plot_multiple_ROCs(y_truths_list,y_scores_list,models=['train','val','test'],title=name)
    plt.savefig(f'results/img/{name}_roc.svg')
    plt.show()

In [ ]:

y_truths_list=[y_train,y_train,y_train]
y_scores_list=[proba_dict_train['LR'],proba_dict_train['MLP'],proba_dict_train['GBM']]
plot_multiple_ROCs(y_truths_list,y_scores_list,models=['LR','MLP','GBM'])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_calibration_curves
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import HosmerLemeshow
for name in proba_dict_test:
    y_truths_list=[y_train,y_val,y_test]
    y_scores_list=[proba_dict_train[name],proba_dict_val[name],proba_dict_test[name]]
    plot_calibration_curves(y_truths_list,y_scores_list,models=['train','val','test'],title=name)
    hl_result_train = HosmerLemeshow(proba_dict_train[name], y_train,Q=5)
    print(f'{name} train:\n{hl_result_train}')
    hl_result_val = HosmerLemeshow(proba_dict_val[name], y_val,Q=5)
    print(f'{name} val:\n {hl_result_val}')
    hl_result_test = HosmerLemeshow(proba_dict_test[name], y_test,Q=5)
    print(f'{name} test:\n {hl_result_test}')
    plt.savefig(f'results/img/{name}_calibrated.svg')
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_DCA_curve

# Example usage:
for names in models:
    intersection_points_x_axis, intersection_points_model_vs_all = plot_DCA_curve(proba_dict_test[names[0]], y_test,model=names[0])
    print("Intersection points with x-axis:", intersection_points_x_axis)
    print("Intersection points between net_benefit_model and net_benefit_all:", intersection_points_model_vs_all)
    plt.savefig(f"results/img/{names[0]}_dca.svg")
    plt.show()

delong 检验

In [ ]:
from itertools import combinations
import pandas as pd
from pixelmed_calc.medical_imaging.RadiologyComponents.delong import delong_roc_test

# List of models and data types
model_names =[]
for name, model in models:
    model_names.append(name)
data_types = ['test','val', 'train']

# Prepare to collect results
results = []

# Note: Assuming y_test_sel and y_train_sel are defined elsewhere in your code
# ground_truth for test and train data
ground_truth_test = y_test
ground_truth_train = y_train

# Generate all combinations of model comparisons for each data type
for sm1, sm2 in combinations(model_names, 2):
    for data_type in data_types:
        predictions_one = pd.read_csv(f'results/pred/{sm1}_{data_type}_proba.csv')
        predictions_two = pd.read_csv(f'results/pred/{sm2}_{data_type}_proba.csv')
        
        # Select appropriate ground_truth based on data_type
        ground_truth = ground_truth_train if data_type == 'train' else ground_truth_test
        
        delong = delong_roc_test(ground_truth, predictions_one['proba'], predictions_two['proba'])
        
        # Collect each comparison's results
        results.append({
            'Model 1': sm1,
            'Model 2': sm2,
            'Data Type': data_type,
            'P-Value': delong[0][0],
            'Z-Value': delong[1][0]
        })

# Convert results into a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
results_df.to_csv('results/delong_test.csv')
results_df
